In [4]:
import pandas as pd

# Read the Parquet file
df = pd.read_parquet("CACHE_DIR/us_quarter_ret.pq")

# Display the first few rows
print(df)

             Date StockID     MarketCap   log_ret        Ret_5d   Ret_20d  \
0      1992-03-31   10001  1.263125e+04 -0.021053  7.362590e-07 -0.021277   
1      1992-06-30   10001  1.263125e+04 -0.031416  4.255318e-02  0.063831   
2      1992-09-30   10001  1.612500e+04  0.025318  3.712644e-07  0.000002   
3      1992-12-31   10001  1.512000e+04 -0.017700  3.571454e-02  0.000001   
4      1993-03-31   10001  1.531875e+04  0.008811 -8.771614e-03  0.105263   
...           ...     ...           ...       ...           ...       ...   
866195 2019-09-30   93436  4.335660e+07 -0.005218 -1.307759e-02  0.360527   
866196 2019-12-31   93436  7.574366e+07  0.008715  1.764403e-01  0.531828   
866197 2020-03-31   93436  9.694000e+07  0.042632  4.093503e-02  0.527692   
866198 2020-06-30   93436  2.008447e+08  0.067478  2.649259e-01  0.388308   
866199 2020-07-31   93436  2.666393e+08 -0.038884  9.711229e-03  0.183757   

         Ret_60d  Ret_6-20d  Ret_6-60d  quarter_ret  next_quarter_ret  \
0 

In [1]:
import os
os.chdir('/Users/ronaldliu/Downloads/jiang_2023/Scripts_2')
os.getcwd()
os.listdir()
import os
from Portfolio import portfolio as pf
import pandas as pd
from Data import equity_data as eqd

In [3]:
import pandas as pd
import os
os.chdir('/Users/ronaldliu/Downloads/jiang_2023/Scripts_2')
# Read the CSV file
df = pd.read_csv("../WORK_SPACE/data/stocks_dataset/raw_data/us_920101-200731.csv", compression='gzip')

# Display the first 5 rows
print(df.head())

/var/folders/m2/zxp_b2wj3td7hjpl_fntxdq80000gn/T/ipykernel_10899/3116691130.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../WORK_SPACE/data/stocks_dataset/raw_data/us_920101-200731.csv", compression='gzip')


   PERMNO        date  EXCHCD  BIDLO   ASKHI     PRC    VOL       RET  SHROUT  \
0   10001  1992-01-02     3.0   14.5  14.500  14.500  100.0  0.000000  1075.0   
1   10001  1992-01-03     3.0   14.5  14.500  14.500  498.0  0.000000  1075.0   
2   10001  1992-01-06     3.0   14.5  14.500  14.500  100.0  0.000000  1075.0   
3   10001  1992-01-07     3.0   14.5  15.250  14.500  417.0  0.000000  1075.0   
4   10001  1992-01-08     3.0   14.5  15.125  15.125  500.0  0.043103  1075.0   

   OPENPRC  
0      NaN  
1      NaN  
2      NaN  
3      NaN  
4      NaN  


In [4]:
df.head(20)

,PERMNO,date,EXCHCD,BIDLO,ASKHI,PRC,VOL,RET,SHROUT,OPENPRC
0,10001,1992-01-02,3.0,14.50,14.500,14.500,100.0,0.000000,1075.0,NaN
1,10001,1992-01-03,3.0,14.50,14.500,14.500,498.0,0.000000,1075.0,NaN
2,10001,1992-01-06,3.0,14.50,14.500,14.500,100.0,0.000000,1075.0,NaN
3,10001,1992-01-07,3.0,14.50,15.250,14.500,417.0,0.000000,1075.0,NaN
4,10001,1992-01-08,3.0,14.50,15.125,15.125,500.0,0.043103,1075.0,NaN
5,10001,1992-01-09,3.0,14.50,14.500,14.500,1200.0,-0.041322,1075.0,NaN
6,10001,1992-01-10,3.0,14.50,14.500,14.500,150.0,0.000000,1075.0,NaN
7,10001,1992-01-13,3.0,14.50,14.500,14.500,1200.0,0.000000,1075.0,NaN
8,10001,1992-01-14,3.0,14.50,14.500,14.500,800.0,0.000000,1075.0,NaN
9,10001,1992-01-15,3.0,14.50,15.250,15.000,1200.0,0.034483,1075.0,NaN
